# Simple Console

In [11]:
from dewloosh.math.linalg.vector import Vector
from dewloosh.math.linalg import linspace, Vector

from dewloosh.mesh.space import StandardFrame, \
    PointCloud, frames_of_lines

from dewloosh.solid.fem import PointData
from dewloosh.solid.fem.cells import B2 as Bernoulli
from dewloosh.solid.fem import LineMesh
from dewloosh.solid.fem import Structure

import numpy as np
from numpy import pi as PI

L = 3000.  # length of the console
R = 25.0  # radius of the tube
nElem = 20  # number of finite elements to use
F = -1000.  # value of the vertical load at the free end
E = 210000.0  # Young's modulus
nu = 0.3  # Poisson's ratio
angles = None
tol = 1e-3

# cross section
A = PI * R**2
Ix = PI * R**4 / 2
Iy = PI * R**4 / 4
Iz = PI * R**4 / 4

# model stiffness matrix
G = E / (2 * (1 + nu))
Hooke = np.array([
    [E*A, 0, 0, 0],
    [0, G*Ix, 0, 0],
    [0, 0, E*Iy, 0],
    [0, 0, 0, E*Iz]
])

# space
GlobalFrame = StandardFrame(dim=3)
if angles is None:
    angles = np.random.rand(3) * PI
local_frame = GlobalFrame.fork('Body', angles, 'XYZ')

# mesh
p0 = np.array([0., 0., 0.])
p1 = np.array([L, 0., 0.])
coords = linspace(p0, p1, nElem+1)
coords = PointCloud(coords, frame=local_frame).show()
topo = np.zeros((nElem, 2), dtype=int)
topo[:, 0] = np.arange(nElem)
topo[:, 1] = np.arange(nElem) + 1

# support at the leftmost, load at the rightmost node
loads = np.zeros((coords.shape[0], 6))
fixity = np.zeros((coords.shape[0], 6)).astype(bool)
global_load_vector = Vector([0., F, F], frame=local_frame).show()
loads[-1, :3] = global_load_vector
fixity[0, :] = True

# pointdata
pd = PointData(coords=coords, frame=GlobalFrame,
               loads=loads, fixity=fixity)

# celldata
frames = frames_of_lines(coords, topo)
cd = Bernoulli(topo=topo, frames=frames)

# set up mesh and structure
mesh = LineMesh(pd, cd, model=Hooke, frame=GlobalFrame)
structure = Structure(mesh=mesh)


In [12]:
# solution
structure.linsolve()


In [13]:
# Bernoulli solution
EI = E * Iy
sol_exact = F * L**3 / (3 * EI)
tol = np.abs(sol_exact / 1000)


In [14]:
# postproc
# 1) displace the mesh
dofsol = structure.nodal_dof_solution()[:, :3]
coords_new = coords + dofsol
structure.mesh.pointdata['x'] = coords_new
local_dof_solution = \
    Vector(dofsol[-1, :3], frame=GlobalFrame).show(local_frame)
sol_fem_Y = local_dof_solution[1]
sol_fem_Z = local_dof_solution[2]


In [15]:
sol_fem_Y, sol_fem_Z, sol_exact


(-139.70156718059263, -139.7015671856567, -139.69256719380073)

In [16]:
from pyvistaqt import BackgroundPlotter
plotter = BackgroundPlotter()
structure.pvplot(plotter=plotter)
plotter.show()
